In [4]:
from pydantic import BaseModel, Field, ValidationError, validator
from typing import Optional

class XUser(BaseModel):
    username: str = Field(..., min_length=3, max_length=15)
    post: str
    account_type: str = Field(..., description="free or premium")

    @validator("username")
    def check_reserved_names(cls, v):
        if v.lower() in ["admin", "twitter"]:
            raise ValueError("Username is reserved and cannot be used.")
        return v

    @validator("account_type")
    def check_account_type(cls, v):
        if v.lower() not in ["free", "premium"]:
            raise ValueError("Account type must be 'free' or 'premium'.")
        return v.lower()

    @validator("post")
    def check_post_length(cls, v, values):
        account = values.get("account_type")
        if not account:
            return v  

        max_len = 280 if account == "free" else 25000
        if len(v) > max_len:
            raise ValueError(f"{account.capitalize()} users can only post up to {max_len} characters.")
        return v


# Test cases to validate the XUser model
def test_users():
    print("=== Running Validation Tests ===")

    users_to_test = [
        {"username": "Admin", "account_type": "free", "post": "Hello X!"},
        {"username": "kenstar", "account_type": "free", "post": "Short post"},
        {"username": "kenny", "account_type": "premium", "post": "A" * 30000},  # too long
    ]

    for data in users_to_test:
        try:
            user = XUser(**data)
            print(f"Valid user: {user.username} ({user.account_type})")
        except ValidationError as e:
            print(f"Validation failed for {data['username']}:")
            print(e, "\n")

test_users()


=== Running Validation Tests ===
Validation failed for Admin:
1 validation error for XUser
username
  Value error, Username is reserved and cannot be used. [type=value_error, input_value='Admin', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error 

Valid user: kenstar (free)
Valid user: kenny (premium)


C:\Users\akind\AppData\Local\Temp\ipykernel_42216\3237092234.py:9: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator("username")
C:\Users\akind\AppData\Local\Temp\ipykernel_42216\3237092234.py:15: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator("account_type")
C:\Users\akind\AppData\Local\Temp\ipykernel_42216\3237092234.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. Yo